# Scrape the Alpha Vantage api and collect daily prices in the backgroung

## Get daily financial data of all companies, going a quarter back

In [2]:
try:
    import alpha_vantage
except:
    ! pip install alpha_vantage
    import alpha_vantage

import requests
from pprint import pprint
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import json
import time
from tqdm import tqdm
import calendar
import os
import logging
# set up logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# create a file handler
handler = logging.FileHandler('price_scrape.log')
handler.setLevel(logging.INFO)
# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(handler)
sns.set()
pd.set_option('display.max_colwidth',1000)

Load all keys for the alpha vantage api and multiply it by 5 since each key is allowed up to 5 requests per minute

In [3]:
with open('../keys.json') as f:
    keys = json.load(f) * 5

## download the dataset for all stocks

Load the CSV of all tickers in the us stock market and use it to get the price of all stocks

## <span style="color:red">As a first step, take all the stock data. Later on for daily data you will take only stocks with highest exchange volume or most mentioned in the daily news</span>

In [4]:
# load pd dataframe from csv
tickers = pd.read_csv('../tickers.csv')
tickers.head()

,symbol,name,exchange,assetType,ipoDate,ipoYear
0,BA,Boeing Company,NYSE,Stock,1962-01-02,1962
1,HWM,Howmet Aerospace Inc,NYSE,Stock,1962-01-02,1962
2,IBM,International Business Machines Corp,NYSE,Stock,1962-01-02,1962
3,KO,Coca-Cola Company,NYSE,Stock,1962-01-02,1962
4,CAT,Caterpillar Inc,NYSE,Stock,1962-01-02,1962


In [ ]:
def get_stockprice(company_symbol: str = '', key: str = 'API_KEY'):
    endpoint = "https://www.alphavantage.co/query"
    parameters = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": company_symbol,
        "outputsize": 'compact',
        "apikey": key,
    }
    # Send a GET request to the API endpoint
    response = requests.get(endpoint, params=parameters)
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        if 'Note' in data: 
            logger.warning(f'API key {key} has been used too many times. Please wait for 1 minute.')
            data = None
    else: 
        logger.error(f'API key {key} has returned an error. response note: {response.json()}')
    return data

# pprint(get_stockprice('MSFT'))

Sweep over all stock tickers and pull daily data

In [ ]:
# get stock price for all tickers
def get_stockprice_all():
    # get a list of all files in the directory without file extension
    seen_stocks = set([f.split('.')[0] for f in os.listdir('prices') if os.path.isfile(os.path.join('prices', f))])
    # only get stock price for stocks that are not in the directory
    unseen_stocks = set(tickers['symbol'].unique()) - seen_stocks
    # create an iterator for keys
    key_iterator = iter(keys)
    for ticker in tqdm(unseen_stocks):
        # get a new key and reset the iterator if it reaches the end
        try:
            key = next(key_iterator)
        except StopIteration:
            time.sleep(60)
            key_iterator = iter(keys)
            key = next(key_iterator)
        
        data = get_stockprice(ticker, key)
        if not data: 
            keys.remove(key)
            # log the remaining keys
            logger.info(f'keys remaining: {len(keys)}')
            continue
        with open(f'prices/{ticker}.json', 'w') as outfile:
            json.dump(data, outfile, indent=4)

# open a new directory to store the data
os.makedirs('prices', exist_ok=True)
get_stockprice_all()

### <span style="color:red">TODO later on, combine the price dataset with the sentiment dataset such that for each stock price you have the daily averaged sentiment as well</span>